<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Phoenix Prompts + OpenAI + Anthropic JavaScript SDK Tutorial</h1>

Let's see how to get started with using the Phoenix JavaScript SDK to pull prompts from an
instance of Phoenix, and then invoke those prompts using multiple LLM SDKs, all in a Deno Jupyter notebook.

> Note: that this example requires an OpenAI API key, an Anthropic API key, and assumes you are running the Phoenix server on localhost:6006.

In [ ]:
import * as PhoenixClient from "npm:@arizeai/phoenix-client"
import OpenAI from "npm:openai"
import { Anthropic } from "npm:@anthropic-ai/sdk"

Let's get started by creating a Phoenix client. This is technically optional, as the Phoenix Prompt helper methods will create a client if not provided.

However, we will create a client here to show how to configure the client with your Phoenix instance.

In [ ]:
const px = PhoenixClient.createClient({
  options: {
    baseUrl: "http://localhost:6006",
    // Uncomment this if you are using a Phoenix instance that requires an API key
    // headers: {
    //   Authorization: "bearer xxxxxx",
    // }
  }
})

Next, let's setup the OpenAI client. This will prompt you for an OpenAI API key.

In [ ]:
const apiKey = prompt("Enter your OpenAI API key:")

In [ ]:
const openai = new OpenAI({ apiKey: apiKey });

We will also setup the Anthropic client, which will prompt you for an Anthropic API key.

In [ ]:
const anthropicApiKey = prompt("Enter your Anthropic API key:")

In [ ]:
const anthropic = new Anthropic({ apiKey: anthropicApiKey });

Now let's build a prompt in Phoenix!

We will create a prompt called `question-searcher` that asks a question and then returns the answer.

Let's set up the prompt to use `OpenAI` as the LLM provider, and lets also add a "variable" to one of the prompt messages, named `query`.

This will allow us to pass in a question to the prompt when we invoke it.

Finally, lets add a tool called `search` to the prompt. This tool will allow the LLM to request a search for a query it receives.

```json
{
  "type": "function",
  "function": {
    "name": "search",
    "description": "Query the internet for the answer to a question.",
    "parameters": {
      "type": "object",
      "properties": {
        "query": {
          "type": "string",
          "description": "Search term"
        }
      },
      "required": ["query"]
    }
  }
}
```

Note that the tool definition is in the format that OpenAI expects, but not Anthropic. We will come back to this later!

[Follow the steps outlined in the documentation](https://docs.arize.com/phoenix/prompt-engineering/how-to-prompts/create-a-prompt) to build your prompt as described above, in the Phoenix UI.

Once you have a prompt, you can pull it into the notebook using the Phoenix client.

In [ ]:
import * as Prompts from "npm:@arizeai/phoenix-client/prompts"

const questionSearcherPrompt = await Prompts.getPrompt({ client: px, prompt: { name: "question-searcher" } })

await Deno.jupyter.md`
  ### question-searcher prompt

  \`\`\`json
  ${JSON.stringify(questionSearcherPrompt, null, 2)}
  \`\`\`
  `

Great! Now we have the contents of the `question-searcher` prompt. It's currently in a format that Phoenix understands, so we need to convert it to a format that our SDKs can understand.

Luckily, `@arizeai/phoenix-client` comes with a helper function, `toSDK`, to convert the prompt to LLM SDK compatible formats, and inject variables into the prompt at the same time.

We will leverage this helper function to convert the prompt to an OpenAI format, and an Anthropic format, and inject the `query` variable into the prompt.

In [ ]:
const openaiPrompt = Prompts.toSDK({ 
  sdk: "openai", 
  prompt: questionSearcherPrompt, 
  variables: { query: "When does the next Arize AI Phoenix release come out?" } 
})

await Deno.jupyter.md`
  ### OpenAI Prompt

  \`\`\`json
  ${JSON.stringify(openaiPrompt, null, 2)}
  \`\`\`
`

In [ ]:
const anthropicPrompt = Prompts.toSDK({ 
  sdk: "anthropic", 
  prompt: questionSearcherPrompt, 
  variables: { query: "When does the next Arize Phoenix release come out?" } 
})

await Deno.jupyter.md`
  ### Anthropic Prompt

  \`\`\`json
  ${JSON.stringify(anthropicPrompt, null, 2)}
  \`\`\`
`

We now have a set of "invocation parameters", fully typed and in the format that OpenAI and Anthropic expect.

Notably, this includes tool definitions as well. The format of the tool definitions is different for OpenAI and Anthropic, and the `@arizeai/phoenix-client` library handles this for you via the `toSDK` function.

Let's invoke OpenAI with our prompt and then Anthropic with our prompt, and compare the results!

First, let's invoke OpenAI with our prompt.

In [ ]:
const response = await openai.chat.completions.create({
  ...openaiPrompt,
  // you can still override any of the invocation parameters as needed
  // for example, you can change the model or stream the response
  model: "gpt-4o-mini",
  stream: false
})

await Deno.jupyter.md`
  ### OpenAI Response

  ${response.choices[0].message.content ?? `\`\`\`json\n${JSON.stringify(response.choices[0].message.tool_calls, null, 2)}\`\`\``}
`

Then, let's invoke Anthropic with our prompt.

In [ ]:
const response = await anthropic.messages.create({
  ...anthropicPrompt,
  // you can still override any of the invocation parameters as needed
  // for example, you can change the model or stream the response
  model: "claude-3-5-sonnet-latest",
  stream: false
})

await Deno.jupyter.md`
  ### Anthropic Response

  ${response.content.map(c => {
    if (c.type === "text") {
      return c.text
    } else if (c.type === "tool_use") {
      return `\`\`\`json\n${JSON.stringify(c, null, 2)}\`\`\``
    }
  }).join("\n")}
`

Great! We've successfully invoked OpenAI, and Anthropic with our prompt and got a response.

You can see that this workflow allows you to build a prompt in Phoenix,
and then invoke it with multiple LLM providers, all without having to modify the prompt or the invocation parameters yourself.

Here are some additional features and capabilities you can explore:

- Prompt Versioning / Tagging
- Prompt Conversion to other LLM providers